In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage import io
from PIL import Image, ImageEnhance
from IPython.display import display
from PIL import Image
import os
import random
import cv2
import time

from tensorflow.keras.applications.inception_v3 import preprocess_input

import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import backend, layers, metrics

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception, VGG19, ResNet50, mobilenet_v2, EfficientNetB3, DenseNet121, VGG16,ConvNeXtTiny,ConvNeXtBase,MobileNet
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import zipfile
import os
import shutil

In [ ]:
# Define paths for input and output data
input_data_path = '/kaggle/input/casiawebface-dataset-crop/CASIA-WebFace_crop'
output_data_path = '/kaggle/working/casia_meglass_datasets'

# Copy or move processed data from input directory to output directory
# Here, we're recursively copying all files and directories
shutil.copytree(input_data_path, output_data_path)

# Confirm that data has been replicated
print("Data replication completed.")

In [ ]:
# Define paths
input_dir = '/kaggle/working/casia_dataset'
output_dir = '/kaggle/working/casia_dataset'

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Group files based on their prefix
file_groups = {}
for file_name in os.listdir(input_dir):
    if os.path.isfile(os.path.join(input_dir, file_name)):
        prefix = file_name[:7]  # Extract the first six characters (0000120_015.jpg) (jhbjh_0004.jpg)
        if prefix not in file_groups:
            file_groups[prefix] = []
        file_groups[prefix].append(file_name)

# Move files to respective folders
for prefix, files in file_groups.items():
    folder_path = os.path.join(output_dir, prefix)
    os.makedirs(folder_path, exist_ok=True)
    c=0
    for file_name in files:
        c+=1
        if c>9 and c<100:
            edit_file = prefix+'_00'+str(c)+'.jpg'
        elif c<=9:
            edit_file = prefix+'_000'+str(c)+'.jpg'
        else:
            edit_file = prefix+'_0'+str(c)+'.jpg'
        source_path = os.path.join(input_dir, file_name)
        destination_path = os.path.join(folder_path, edit_file)
        shutil.move(source_path, destination_path)

print("Files moved successfully.")

In [ ]:
# Define paths for input and output data
input_data_path = '/kaggle/input/meglass/MeGlass_ori'
output_data_path = '/kaggle/working/MeGlass'

# Copy or move processed data from input directory to output directory
# Here, we're recursively copying all files and directories
shutil.copytree(input_data_path, output_data_path)

# Confirm that data has been replicated
print("Data replication completed.")

In [ ]:
# Define paths
import re
input_dir = '/kaggle/working/MeGlass'
output_dir = '/kaggle/working/MeGlass'
# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Group files based on their prefix
file_groups = {}
for file_name in os.listdir(input_dir):
    if os.path.isfile(os.path.join(input_dir, file_name)):
        prefix= file_name.split("@")[0]
        #prefix = file_name[:7]  # Extract the first six characters (0000120_015.jpg) (jhbjh_0004.jpg)
        if prefix not in file_groups:
            file_groups[prefix] = []
        file_groups[prefix].append(file_name)
        
for prefix, files in file_groups.items():
    folder_path = os.path.join(output_dir, prefix)
    os.makedirs(folder_path, exist_ok=True)
    c=0
    for file_name in files:
        c+=1
        if c>9 and c<100:
            edit_file = prefix+'_00'+str(c)+'.jpg'
        elif c<=9:
            edit_file = prefix+'_000'+str(c)+'.jpg'
        else:
            edit_file = prefix+'_0'+str(c)+'.jpg'
        source_path = os.path.join(input_dir, file_name)
        destination_path = os.path.join(folder_path, edit_file)
        shutil.move(source_path, destination_path)

print("Files moved successfully.")

In [ ]:
import os
import shutil
 
# Define the paths to the source and destination folders
source_folder ='/kaggle/working/MeGlass'
destination_folder ='/kaggle/working/casia_meglass_datasets'
 
# Get a list of folders inside the source_folder
folders_to_move = [f for f in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, f))]
 
# Move each folder to the destination folder
for folder_name in folders_to_move:
    source_path = os.path.join(source_folder, folder_name)
    destination_path = os.path.join(destination_folder, folder_name)
    shutil.move(source_path, destination_path)
print("Folders moved successfully.")

In [ ]:
# Setting random seeds to enable consistency while testing.
random.seed(5)
np.random.seed(5)
tf.random.set_seed(5)

ROOT = r"/kaggle/working/casia_meglass_datasets"

def read_image(index):
#     path = os.path.join(ROOT, index[0], index[1])
    if(index[1]>9 and index[1]<100):
        path = os.path.join(ROOT, index[0], index[0]+'_00'+str(index[1])+'.jpg')
    elif(index[1]<=9):
        path = os.path.join(ROOT, index[0], index[0]+'_000'+str(index[1])+'.jpg')
    else:
        path = os.path.join(ROOT, index[0], index[0]+'_0'+str(index[1])+'.jpg')
    
    image = cv2.imread(path)
    #print(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(Image.fromarray(image).resize((128, 128)))
    #image = image.reshape(image.shape[0], 128, 128, 3)
    
    return image

In [ ]:
def split_dataset(directory, split=0.9):
    folders = os.listdir(directory)
    num_train = int(len(folders)*split)
    
    random.shuffle(folders)
    
    train_list, test_list = {}, {}
    
    # Creating Train-list
    for folder in folders[:num_train]:
        num_files = len(os.listdir(os.path.join(directory, folder)))
        train_list[folder] = num_files
    
    # Creating Test-list
    for folder in folders[num_train:]:
        num_files = len(os.listdir(os.path.join(directory, folder)))
        test_list[folder] = num_files  
    
    return train_list, test_list

train_list, test_list = split_dataset(ROOT, split=0.9)
print("Length of training list:", len(train_list))
print("Length of testing list :", len(test_list))

# train_list, test list contains the folder names along with the number of files in the folder.
print("\nTest List:", test_list)

In [ ]:
def create_triplets(directory, folder_list, max_files=4):
    triplets = []
    folders = list(folder_list.keys())

    for folder in folders:
        path = os.path.join(directory, folder)
        files = list(os.listdir(path))[:max_files]
        num_files = len(files)

        for i in range(1,num_files):
            for j in range(i+1, num_files):
                anchor = (folder, i)
                positive = (folder, j)

                neg_folder = folder
                while neg_folder == folder:
                    neg_folder = random.choice(folders)
                neg_file = random.randint(1, folder_list[neg_folder])
                negative = (neg_folder, neg_file)

                triplets.append((anchor, positive, negative))

    random.shuffle(triplets)
    return triplets

In [ ]:
train_triplet = create_triplets(ROOT, train_list)
test_triplet  = create_triplets(ROOT, test_list)

print("Number of training triplets:", len(train_triplet))
print("Number of testing triplets :", len(test_triplet))

print("\nExamples of triplets:")
for i in range(10):
    print(train_triplet[i])

In [ ]:
from PIL import Image
def get_batch(triplet_list, batch_size=256, preprocess=True):
    batch_steps = len(triplet_list)//batch_size

    for i in range(batch_steps+1):
        anchor   = []
        positive = []
        negative = []

        j = i*batch_size
        while j<(i+1)*batch_size and j<len(triplet_list):
            a, p, n = triplet_list[j]
            anchor.append(read_image(a))
            positive.append(read_image(p))
            negative.append(read_image(n))
            j+=1

        anchor = np.array(anchor)
        positive = np.array(positive)
        negative = np.array(negative)

        #print("Original shapes:", anchor.shape, positive.shape, negative.shape)

        # Resize images to (128, 128)
#         anchor_resized = np.array([np.array(Image.fromarray(img).resize((128, 128))) for img in anchor])
#         positive_resized = np.array([np.array(Image.fromarray(img).resize((128, 128))) for img in positive])
#         negative_resized = np.array([np.array(Image.fromarray(img).resize((128, 128))) for img in negative])

#         # Reshape
#         anchor = anchor_resized.reshape(anchor_resized.shape[0], 128, 128, 3)
#         positive = positive_resized.reshape(positive_resized.shape[0], 128, 128, 3)
#         negative = negative_resized.reshape(negative_resized.shape[0], 128, 128, 3)
        #print("Reshaped shapes:", anchor.shape, positive.shape, negative.shape)

        if preprocess:
            anchor = preprocess_input(anchor)
            positive = preprocess_input(positive)
            negative = preprocess_input(negative)

        yield ([anchor, positive, negative])

In [ ]:
import matplotlib. pyplot as plt
num_plots = 10

f, axes = plt.subplots(num_plots, 3, figsize=(15, 20))

for x in get_batch(train_triplet, batch_size=num_plots, preprocess=False):
    a,p,n = x
    for i in range(num_plots):
        axes[i, 0].imshow(a[i])
        axes[i, 1].imshow(p[i])
        axes[i, 2].imshow(n[i])
        i+=1
    break

In [ ]:
def get_encoder(input_shape):
    """ Returns the image encoding model """

    pretrained_model =Xception(
        input_shape=input_shape,
        weights='imagenet',
        include_top=False,
        pooling='avg',
    )

    for i in range(len(pretrained_model.layers)-27):
        pretrained_model.layers[i].trainable = False

    encode_model = Sequential([
        pretrained_model,
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(256, activation="relu"),
        layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
    ], name="Encode_Model")
    return encode_model

In [ ]:
class DistanceLayer(layers.Layer):
    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)


def get_siamese_network(input_shape = (128, 128, 3)):
    encoder = get_encoder(input_shape)

    # Input Layers for the images
    anchor_input   = layers.Input(input_shape, name="Anchor_Input")
    positive_input = layers.Input(input_shape, name="Positive_Input")
    negative_input = layers.Input(input_shape, name="Negative_Input")

    ## Generate the encodings (feature vectors) for the images
    encoded_a = encoder(anchor_input)
    encoded_p = encoder(positive_input)
    encoded_n = encoder(negative_input)

    # A layer to compute ‖f(A) - f(P)‖² and ‖f(A) - f(N)‖²
    distances = DistanceLayer()(
        encoder(anchor_input),
        encoder(positive_input),
        encoder(negative_input)
    )

    # Creating the Model
    siamese_network = Model(
        inputs  = [anchor_input, positive_input, negative_input],
         outputs = distances,
        name = "Siamese_Network"
    )
    return siamese_network

siamese_network = get_siamese_network()
siamese_network.summary()

In [ ]:
plot_model(siamese_network, show_shapes=True, show_layer_names=True)

In [ ]:
class SiameseModel(Model):
    # Builds a Siamese model based on a base-model
    def __init__(self, siamese_network, margin=1.4):
        super(SiameseModel, self).__init__()

        self.margin = margin
        self.siamese_network = siamese_network
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape get the gradients when we compute loss, and uses them to update the weights
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.siamese_network.trainable_weights))

        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}
    def _compute_loss(self, data):
        # Get the two distances from the network, then compute the triplet loss
        ap_distance, an_distance = self.siamese_network(data)
        loss = tf.maximum(ap_distance - an_distance + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics so the reset_states() can be called automatically.
        return [self.loss_tracker]

In [ ]:
siamese_model = SiameseModel(siamese_network)

optimizer = Adam(learning_rate=1e-3, epsilon=1e-01)
siamese_model.compile(optimizer=optimizer)

In [ ]:
def test_on_triplets(batch_size = 256):
    pos_scores, neg_scores = [], []

    for data in get_batch(test_triplet, batch_size=batch_size):
        prediction = siamese_model.predict(data)
        pos_scores += list(prediction[0])
        neg_scores += list(prediction[1])

    accuracy = np.sum(np.array(pos_scores) < np.array(neg_scores)) / len(pos_scores)
    ap_mean = np.mean(pos_scores)
    an_mean = np.mean(neg_scores)
    ap_stds = np.std(pos_scores)
    an_stds = np.std(neg_scores)

#   print(f"Accuracy on test = {accuracy:.5f}")
    return (accuracy, ap_mean, an_mean, ap_stds, an_stds)

In [ ]:
save_all = False
epochs = 10
batch_size = 128

max_acc = 0
train_loss = []
test_metrics = []
accuracy_list = []
counter=0

for epoch in range(1, epochs+1):
    t = time.time()
    # Training the model on train data
    epoch_loss = []
    for data in get_batch(train_triplet, batch_size=batch_size):
        loss = siamese_model.train_on_batch(data)
        epoch_loss.append(loss)
    epoch_loss = sum(epoch_loss)/len(epoch_loss)
    train_loss.append(epoch_loss)

    print(f"\nEPOCH: {epoch} \t (Epoch done in {int(time.time()-t)} sec)")
    print(f"Loss on train    = {epoch_loss:.5f}")
    
    # Testing the model on test data
    metric = test_on_triplets(batch_size=batch_size)
    test_metrics.append(metric)
    accuracy = metric[0]
    print(f"Accuracy    = {accuracy}")
    
    # Saving the model weights
    if save_all or accuracy>=max_acc:
        siamese_model.save_weights("siamese_model")
        max_acc = accuracy
    accuracy_list = np.append(accuracy_list,accuracy)
    if epoch != 1:
        if accuracy_list[epoch-1] < accuracy_list[epoch-2] and counter<3:
            counter+=1
    if counter>=3:
        break

# Saving the model after all epochs run
#siamese_model.save_weights("siamese_model-final")
#siamese_model.save('kaggle/working/VGG16_model.tf')

In [ ]:
def extract_encoder(model):
    encoder = get_encoder((128, 128, 3))
    i=0
    for e_layer in model.layers[0].layers[3].layers:
        layer_weight = e_layer.get_weights()
        encoder.layers[i].set_weights(layer_weight)
        i+=1
    return encoder

encoder = extract_encoder(siamese_model)
encoder.save_weights("VGG16encoder_lfw_casia_without_aug_delete1_max4_100epochs_early_printtrain")
encoder.summary()

In [ ]:
def classify_images(face_list1, face_list2, threshold=1.1):
    # Getting the encodings for the passed faces
    tensor1 = encoder.predict(face_list1)
    tensor2 = encoder.predict(face_list2)
    
    distance = np.sum(np.square(tensor1-tensor2), axis=-1)
    prediction = np.where(distance<=threshold, 0, 1)
    print(distance)
    return prediction

In [ ]:
def ModelMetrics(pos_list, neg_list):
    true = np.array([0]*len(pos_list)+[1]*len(neg_list))
    pred = np.append(pos_list, neg_list)
    
    # Compute and print the accuracy
    print(f"\nAccuracy of model: {accuracy_score(true, pred)}\n")
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(true, pred)

    categories  = ['Similar','Different']
    names = ['True Similar','False Similar', 'False Different','True Different']
    percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten() / np.sum(cf_matrix)]

    labels = [f'{v1}\n{v2}' for v1, v2 in zip(names, percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot = labels, cmap = 'Blues',fmt = '',
                xticklabels = categories, yticklabels = categories)

    plt.xlabel("Predicted", fontdict = {'size':14}, labelpad = 10)
    plt.ylabel("Actual"   , fontdict = {'size':14}, labelpad = 10)
    plt.title ("Confusion Matrix", fontdict = {'size':18}, pad = 20)

In [ ]:
pos_list = np.array([])
neg_list = np.array([])
zero_list = np.array([])

for data in get_batch(test_triplet, batch_size=256):
    a, p, n = data
    pos_list = np.append(pos_list, classify_images(a, p))
    neg_list = np.append(neg_list, classify_images(a, n))

# for data in get_batch(test_triplet, batch_size=256):
#     a, p, n = data
#     pos_list = np.append(pos_list, classify_without_threshold(a, p,n))
#     print(pos_list)
# for i in pos_list:
#     if i==0:
#         zero_list = np.append(zero_list, i)    
# print(len(zero_list))
# print(len(pos_list))
ModelMetrics(pos_list, neg_list)

In [ ]:
# pos_list_train = np.array([])
# neg_list_train = np.array([])
# for i in range(50):
#     a,p,n = train_triplet[i]
#     pos_list_train = np.append(pos_list_train, classify_images(a, p))
#     neg_list_train = np.append(neg_list_train, classify_images(a, n))
pos_list_train = np.array([])
neg_list_train = np.array([])
for data in get_batch(train_triplet, batch_size=256):
    a, p, n = data
    pos_list_train = np.append(pos_list_train, classify_images(a, p))
    neg_list_train = np.append(neg_list_train, classify_images(a, n))
#     pos_list_train = np.append(pos_list_train, classify_without_threshold(a, p,n))
#     print(pos_list_train)
#     print('------------------------------------')
#     print(neg_list_train)
# acc2 = np.array([0]*len(pos_list_train))
# print(len(acc2))
# print(len(pos_list_train))
ModelMetrics(pos_list_train, neg_list_train)

In [ ]:
#siamese_model.save("VGG16_model_lfw_casia_meglass_without_aug_delete1_max4_10epochs_early.h5")
#siamese_model.save('kaggle/working/VGG16_model.tf')
encoder.save("Xception_encoder_lfw_casia_meglass_without_aug_delete1_max4_10epochs_early_margin14.h5")
!zip -r Xception_encoder_lfw_casia_meglass_without_aug_delete1_max4_10epochs_early_margin14.zip /kaggle/working/Xception_encoder_lfw_casia_meglass_without_aug_delete1_max4_10epochs_early_margin14.h5
%cd /kaggle/working/Xception_encoder_lfw_casia_meglass_without_aug_delete1_max4_10epochs_early_margin14.zip
from IPython.display import FileLink
FileLink(r'Xception_encoder_lfw_casia_meglass_without_aug_delete1_max4_10epochs_early_margin14.zip')

In [ ]:
pos_list_own_images = np.array([])
neg_list_own_images = np.array([])
anchor   = []
positive = []
negative = []
positive_distance_list=[]
names_positive_list = []
negative_distance_list=[]
names_negative_list = []
threshold=0.1
#student_image = np.expand_dims(student_image, axis=0)
def read_our_image(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(Image.fromarray(image).resize((128, 128)))
    
    face_cascade = cv2.CascadeClassifier('/kaggle/input/haarcascade/haarcascade_frontalface_default 1.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.02, minNeighbors=5)
    #print(np.shape(faces))
    try:
        x,y,w,h=faces[0]
        face_img = image[y:y+h, x:x+w]
#         plt.imshow(face_img)
#         plt.axis('off')  # Turn off axis
#         plt.show()
    except:
        face_img=image
#         print('********************************************')
#         plt.imshow(face_img)
#         plt.axis('off')  # Turn off axis
#         plt.show()
    return face_img
anchor_folder = '/kaggle/input/our-images/Nancy'
anchor_file= 'Nancy2.jpg'
anchor_path = os.path.join(anchor_folder, anchor_file)
anchor.append(read_our_image(anchor_path))
our_images_path = '/kaggle/input/our-images'
# List all folders in the dataset directory and get the first three
folders = os.listdir(our_images_path)
# Loop through the first three folders
for folder in folders:
    folder_path = os.path.join(our_images_path, folder)
    if folder_path == anchor_folder:
        for filename in os.listdir(folder_path):
            positive_path = os.path.join(folder_path, filename)
            names_positive_list = np.append(names_positive_list,filename)
            image = read_our_image(positive_path)
            image = np.expand_dims(image, axis=0)
            image = np.array(image)
            image = preprocess_input(image)
            positive.append(image)
    else:
        for filename in os.listdir(folder_path):
            negative_path = os.path.join(folder_path, filename)
            names_negative_list = np.append(names_negative_list,filename)
            image = read_our_image(negative_path)
            image = np.expand_dims(image, axis=0)
            image = np.array(image)
            image = preprocess_input(image)
            negative.append(image)
anchor = np.array(anchor)
anchor = preprocess_input(anchor)

#pos_list_own_images = np.append(pos_list_train, classify_images(anchor, positive))
#neg_list_own_images = np.append(neg_list_train, classify_images(anchor, negative))
tensor1 = encoder.predict(anchor)
for i in positive:
    tensor2 = encoder.predict(i)
    pos_distance = np.sum(np.square(tensor1-tensor2), axis=-1)
    positive_distance_list = np.append(positive_distance_list, pos_distance)
    pos_prediction = np.where(pos_distance<=threshold, 0, 1)
    pos_list_own_images = np.append(pos_list_own_images, pos_prediction)
for i in negative:
    tensor3 = encoder.predict(i)
    neg_distance = np.sum(np.square(tensor1-tensor3), axis=-1)
    negative_distance_list = np.append(negative_distance_list, neg_distance)
    neg_prediction = np.where(neg_distance<=threshold, 0, 1)
    neg_list_own_images = np.append(neg_list_own_images, neg_prediction)
print(names_positive_list)
print('----------------------------------------')
print(positive_distance_list)
print('----------------------------------------')
print(pos_list_own_images)
print('########################################')
print(names_negative_list)
print('----------------------------------------')
print(negative_distance_list)
print('----------------------------------------')
print(neg_list_own_images)
# tensor2 = encoder.predict(positive)
# tensor3 = encoder.predict(negative)
ModelMetrics(pos_list_own_images, neg_list_own_images)